In [1]:
import sys
from pathlib import Path

# Add user_data directory to path for crypto_analysis package
user_data_path = Path.cwd().parent
data_dir = user_data_path / "data" / "binance"
sys.path.insert(0, str(user_data_path))

In [2]:
from crypto_analysis import (
    DatasetBuilder,
    LSTMMetaheuristicOptimizer,
)


In [3]:
# 1. Build dataset
builder = DatasetBuilder(data_dir=data_dir, period_hours=4, n_workers=15, signal_shift=0)
df = builder.build('DOGE', 
                   threshold_pct=1.5,
                   hyperopt=False,
                   verbose=True)

Generating target signals for DOGE...
Initial index (0): entry=140, exit=140
Found optimal index (0): entry=140, exit=140
Processing 57 indicators with 15 workers...
  [RSI] Grid search optimization...
  [MACD] Grid search optimization...
  [STOCH] Grid search optimization...
  [STOCHRSI] Grid search optimization...
  [CCI] Grid search optimization...
  [MFI] Grid search optimization...
  [WILLR] Grid search optimization...
  [CMO] Grid search optimization...
  [ADX] Grid search optimization...
  [MOM] Grid search optimization...
  [ROC] Grid search optimization...
  [TRIX] Grid search optimization...
  [ULTOSC] Grid search optimization...
  [APO] Grid search optimization...
  [PPO] Grid search optimization...
  [TRIX] Grid search done - score: 16
  [BOP] Grid search optimization...
Completed 1/57: TRIX
  [BOP] Grid search done - score: 0
  [AROON] Grid search optimization...
Completed 2/57: BOP
  [MOM] Grid search done - score: 48
  [AROONOSC] Grid search optimization...
Completed 3/5

c:\Users\irisbridge\Desktop\projects\freqtrade\user_data\crypto_analysis\indicator_optimizer\dataset_builder.py:385: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{ind_name}_gs_exit"] = result.gs_exit
c:\Users\irisbridge\Desktop\projects\freqtrade\user_data\crypto_analysis\indicator_optimizer\dataset_builder.py:384: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{ind_name}_gs_entry"] = result.gs_entry
c:\Users\irisbridge\Desktop\projects\freqtrade\user_data\crypto_analysis\indicator_optimizer\dataset_builder.py:385: Per

In [4]:
df.to_csv("doge.csv", index=False)

In [4]:
import pandas as pd
df = pd.read_csv("doge.csv")

In [ ]:
# # 2. Preprocess
# preprocessor = DataPreprocessor(target_shift=4)
# features, targets = preprocessor.fit_transform(df)

# # 3. Create sequences
# feat_seqs, tgt_seqs = create_sequences(features, targets, input_seq_length=12, output_seq_length=4)

# 4. Validate and filter
# validator = SequenceValidator()
# feat_seqs, tgt_seqs, seq_types = validator.filter_valid_sequences(feat_seqs, tgt_seqs)

# # 5. Create dataset
# dataset = SignalDataset(feat_seqs, tgt_seqs, seq_types)

# # 6. Train
# config = TrainingConfig(
#     epochs=100,
#     auto_class_weights=True,
#     class_weight_power=1.5,     # Even stronger (was 1.0)
#     focal_loss=True,
#     focal_gamma=4.0,            # More aggressive (was 3.0)
#     label_smoothing=0.15,
#     dropout=0.5,
#     weight_decay=0.01,          # Stronger L2
#     hidden_size=64,             # Smaller model in ModelConfig
#     patience=10,
#     checkpoint_dir='checkpoints/'
# )

# # Also use smaller model
# model_config = ModelConfig(input_size=preprocessor.get_num_features(), hidden_size=64, num_layers=1)


# model = LSTMSignalPredictor(model_config)
# trainer = Trainer(model, config, preprocessor=preprocessor)
# history = trainer.train(dataset)


# # Or evaluate silently and print later
# metrics = trainer.evaluate_all()
# trainer.print_evaluation_report(metrics)

# from crypto_analysis import Predictor

# # Load your trained model
# predictor = Predictor.from_checkpoint(
#     'checkpoints/best_model.pt',
#     'checkpoints/preprocessor.pkl'
# )

# # Find optimal threshold on validation data
# threshold_results = predictor.find_optimal_threshold(
#     df=df.tail(int(df.shape[0]*0.2)),
#     thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#     metric='f1',  # or 'precision', 'recall'
#     verbose=True
# )

# print(f"Best threshold: {threshold_results['best_threshold']}")

# # Evaluate performance with thresholded predictions
# metrics = predictor.evaluate_with_threshold(
#     df.tail(int(df.shape[0]*0.2)),
#     entry_threshold=0.7,
#     exit_threshold=0.8
# )

# print(f"Entry Precision: {metrics['entry_precision']:.4f}")
# print(f"Entry Recall: {metrics['entry_recall']:.4f}")
# print(f"Entry F1: {metrics['entry_f1']:.4f}")

# print(f"Exit Precision: {metrics['exit_precision']:.4f}")
# print(f"Exit Recall: {metrics['exit_recall']:.4f}")
# print(f"Exit F1: {metrics['exit_f1']:.4f}")

In [5]:
# Optimize
optimizer = LSTMMetaheuristicOptimizer(
    df=df,
    pop_size=12,              # +2: Better exploration
    iterations=70,            # +20: Still improving, allow more time
    n_workers=15,
    np_neighbors=2,           # +1: Better neighborhood effect
    pf_max=0.18,              # +0.03: More exploration (params diverse)
    epochs_per_eval=100,
    checkpoint_interval=5,
    elitist_selection=True,
    elitist_constant=0.28,    # -0.02: Slightly more exploration
    verbose=True,
    enable_logging=True,
)

result = optimizer.optimize()

print(f"Best fitness: {result.best_fitness}")
print(f"Selected features ({result.n_features_selected}): {result.selected_features}")
print(f"Best params: {result.best_params}")


LSTMMetaheuristicOptimizer (APO) initialized:
  - DataFrame mode: binary
  - Feature columns: 119
  - Hyperparameters: 11
  - Total dimension: 130
  - Population size: 12
  - Iterations: 70
  - Workers: 15
  - APO np_neighbors: 2
  - APO pf_max: 0.18
  - Elitist selection: True
  - Elitist constant: 0.28
  - Correlation vector: min=0.0000, max=0.1226, mean=0.0224

Starting APO Metaheuristic Optimization
Run ID: a2dc93ba

Evaluating initial population...
iter:-1 indv:2 fitness:0.2818 features:56
iter:-1 indv:11 fitness:0.4102 features:66
iter:-1 indv:5 fitness:0.3538 features:59
iter:-1 indv:9 fitness:0.4032 features:58
iter:-1 indv:7 fitness:0.1678 features:62
iter:-1 indv:0 fitness:0.3897 features:52
iter:-1 indv:8 fitness:0.3361 features:61
iter:-1 indv:10 fitness:0.3772 features:63
iter:-1 indv:6 fitness:0.3735 features:59
iter:-1 indv:3 fitness:0.4024 features:51
iter:-1 indv:4 fitness:0.3790 features:62
iter:-1 indv:1 fitness:0.3469 features:57

-----------------------------------

KeyboardInterrupt: 

In [ ]:
trainer = optimizer.train_from_result(result, epochs=100)

In [ ]:
# # Load your trained model
# predictor = Predictor.from_checkpoint(
#     'checkpoints/best_model.pt',
#     'checkpoints/preprocessor.pkl'
# )

# # Find optimal threshold on validation data
# threshold_results = predictor.find_optimal_threshold(
#     df=df.tail(int(df.shape[0]*0.2)),
#     thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#     metric='f1',  # or 'precision', 'recall'
#     verbose=True
# )

# print(f"Best threshold: {threshold_results['best_threshold']}")



In [ ]:
# # Evaluate performance with thresholded predictions
# metrics = predictor.evaluate_with_threshold(
#     df.tail(int(df.shape[0]*0.2)),
#     entry_threshold=0.3,
#     exit_threshold=0.3
# )

# print(f"Entry Precision: {metrics['entry_precision']:.4f}")
# print(f"Entry Recall: {metrics['entry_recall']:.4f}")
# print(f"Entry F1: {metrics['entry_f1']:.4f}")

# print(f"Exit Precision: {metrics['exit_precision']:.4f}")
# print(f"Exit Recall: {metrics['exit_recall']:.4f}")
# print(f"Exit F1: {metrics['exit_f1']:.4f}")

In [ ]:
# predictor.predict_with_threshold(df.tail(int(df.shape[0]*0.2)), 0.9, 0.9).labels

In [ ]:
from crypto_analysis.signal_population import SignalPopulator

# Test with DOGE and period_hours=4
populator = SignalPopulator(data_dir=data_dir, period_hours=4)
df_signals = populator.populate_signals("DOGE", threshold_pct=1.5)

print(f"\nDataFrame shape: {df_signals.shape}")
print(f"Entry signals: {(df_signals['signal'] == 'entry').sum()}")
print(f"Exit signals: {(df_signals['signal'] == 'exit').sum()}")